In [ ]:
from ypath import ypath_system, ppxml

# set of evpn/vxlan related ypaths
ypath_vlan = "/bd-items/bd-items/BD-list[]/fabEncap=vlan-{vlan_id}"
ypath_svi = "/intf-items/svi-items/If-list[]/id=vlan{vlan_id}"
ypath_ipv4 = "/ipv4-items/inst-items/dom-items/Dom-list/name={vrf_name}/if-items/If-list[]/id=vlan{vlan_id}"
ypath_svianycast = "/hmm-items/fwdinst-items/if-items/FwdIf-list[]/id=vlan{vlan_id}"
ypath_nve = "/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vni}"
ypath_evpn = "/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vni}"

ypath_vlan_conf = ypath_vlan + "/name={vlan_name}/id={vlan_id}/BdState=active/adminSt=active/bridgeMode=mac/fwdCtrl=mdst-flood/fwdMode=bridge,route/mode=CE"
ypath_vxlan_conf = ypath_vlan + "/name={vlan_name}/id={vlan_id}/accEncap=vxlan-{vni}/BdState=active/adminSt=active/bridgeMode=mac/fwdCtrl=mdst-flood/fwdMode=bridge,route/mode=CE"
ypath_svi_conf = ypath_svi + "/mtu={mtu}/descr={description}/adminSt=up/rtvrfMbr-items/tDn=//System//inst-items//Inst-list[name='{vrf_name}']"
ypath_ipv4_conf = ypath_ipv4 + "/addr-items/Addr-list/addr={ip_address}"
ypath_svianycast_conf = ypath_svianycast + "/adminSt=enabled/mode=anycastGW"
ypath_nve_mgroup_conf = ypath_nve + "/mcastGroup={mgroup}"
ypath_nve_ir_conf = ypath_nve + "/IngRepl-items/proto=bgp"

ypath_evpn_conf = ypath_evpn + "/rd=rd:unknown:0:0/rttp-items/RttP-list/type=export/ent-items/RttEntry-list/rtt=route-target:unknown:0:0"
# ypath2xml() can't create xml with multiple leafs so we have to add 2nd list separately 
# note: there is no '[]' hooks in the ypath so that the default operation "merge" will be used
ypath_evpn_rtimport_conf = "/evpn-items/bdevi-items/BDEvi-list/encap=vxlan-{vni}/rd=rd:unknown:0:0/rttp-items/RttP-list/type=import/ent-items/RttEntry-list/rtt=route-target:unknown:0:0"


In [ ]:

vlan_id = 300
vlan_name = "vlan300"
vni = 10300
mtu = 9000
description = "blablab"
vrf_name = "Tenant-1"
ip_address = '10.0.1.2//24'
mgroup = '231.0.0.9'

get_list = [ypath_vlan.format(vlan_id=vlan_id),
                ypath_svi.format(vlan_id=vlan_id),
                ypath_ipv4.format(vlan_id=vlan_id, vrf_name=vrf_name),
                ypath_svianycast.format(vlan_id=vlan_id),
                ypath_nve.format(vni=vni),
                ypath_evpn.format(vni=vni)]

conf_list = [ypath_vxlan_conf.format(vlan_id=vlan_id, vlan_name=vlan_name, vni=vni),
                ypath_svi_conf.format(vlan_id=vlan_id, mtu=mtu, description=description, vrf_name=vrf_name),
                ypath_ipv4_conf.format(vlan_id=vlan_id, vrf_name=vrf_name, ip_address=ip_address), 
                ypath_svianycast_conf.format(vlan_id=vlan_id),
                ypath_nve_mgroup_conf.format(vni=vni, mgroup=mgroup),
                ypath_evpn_conf.format(vni=vni),
                ypath_evpn_rtimport_conf.format(vni=vni)]

get_xml = ypath_system(get_list)           

delete_xml = "<config>" + ypath_system(get_list, operation="remove") + "</config>"           

conf_xml = "<config>" + ypath_system(conf_list, operation="replace") + "</config>"
     
              

In [ ]:
#ppxml(get_xml)
#ppxml(conf_xml)
#ppxml(delete_xml)

In [ ]:
from scrapli_netconf.driver import NetconfScrape

def config_scrapli_netconf(conn, config, datastore='running'):    
    response = conn.edit_config(config=config, target=datastore)
    return response

def get_scrapli_netconf(conn, filter_):
    response = conn.get(filter_=filter_, filter_type="subtree")
    return response


my_device = {
    "host": "192.168.99.91",
    "auth_username": "apiuser",
    "auth_password": "apipassword",
    "auth_strict_key": False,        
}


In [ ]:
with NetconfScrape(**my_device) as conn:
    print("************** creating evpn ******************")

    resp = config_scrapli_netconf(conn, conf_xml)    
    print(f"{resp.result = }")

    print("\n*************** check evpn *********************")
    
    resp = get_scrapli_netconf(conn, get_xml)
    print(resp.result)
    
    

In [ ]:
with NetconfScrape(**my_device) as conn:
    print("\n*************** delete evpn *********************")
    
    resp = config_scrapli_netconf(conn, delete_xml)            
    print(resp.result)

    print("\n*************** check evpn again *********************")
    
    resp = get_scrapli_netconf(conn, get_xml)
    print(resp.result)